<img src="../img/Logo.png" width="300">

# Aprendizaje por Imágenes
# Comprimir una imagen usando Machine Learning

### Profesor: Jorge Calvo



El objetivo de esta actividad es demostrar como cn técicas de **Machine Learning** podemos trabajar con imágenes y reducir el su tamaño gracias a métodos de aprendizaje no supervisado.
Para ello hemos usado el método de **K-Means**, un algoritmo de clasificación.

### Algoritmo de K-Means 

* **Procedimiento** <br>
Simplemente reducimos el número total de colores utilizados para representar la imagen, y de esta forma permitimos que se necesiten menos memoria (bits) para su almacenaje.
Buscamos la similitud entre pixeles por medio de la creación de **clusters**, una vez que tengamos esa cercanía entre colores cambiaremos el color de todo ese cluster tomando como referencia el color del centroide.

**Centroide: Pixel central del cluster.**
Cargamos las librerias necesarias para la actividad

In [ ]:
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import tensorflow as tf
from tensorflow.keras.utils import plot_model

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Solo mostrar errores y advertencias críticas

# Configurar el crecimiento de memoria en la GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
print("GPUs disponibles: ", tf.config.experimental.list_physical_devices('GPU'))


#%matplotlib widget
#%matplotlib inline
#%matplotlib qt

Usamos una imagen para realizar la actividad. La imágen se otiene usando la libreria **openCV**

In [ ]:
image = cv2.imread("../images/camion.jpg")
cuadro=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
ax = plt.axes(xticks=[], yticks=[])
ax.imshow(cuadro);

In [ ]:
# Cargar la imagen
image_path = '../images/camion.jpg'
cuadro = plt.imread(image_path)
plt.axis("off")
plt.imshow(cuadro)

### ¿Cómo se representa una imagen?
Recordamos que las imagenes se representan como matrices de `(alto, ancho, canales)`, donde los valores de los canales son rojo/verde/azul y varían de 0 a 255.

In [ ]:
cuadro.shape

Sin embargo, podemos ver este dataset como una nube de puntos tridimensional, donde cada pixel es una instancia.

Vamos a normalizar los valores entre 0 y 1 y a convertirlos en `[n_instancias, 3]`. Cada punto de 3 coordenadas se convierte en un valor.

In [ ]:
# normalizamos la imagen para que tenga valores entre 0 y 1
data = cuadro / 255.
data = data.reshape(cuadro.shape[0] * cuadro.shape[1], 3)
data.shape

In [ ]:
# Definir la red convolucional
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(3,(3, 3),  activation='relu', padding='same', input_shape=cuadro.shape),
    tf.keras.layers.Reshape((cuadro.shape[0]*cuadro.shape[1],3))
])
plot_model(model, to_file='./red_convolucional.png', show_shapes=True, show_layer_names=True, dpi=96)

In [ ]:
# Convertir la imagen a un tensor de TensorFlow
image_tensor = tf.convert_to_tensor(cuadro, dtype=tf.float32)
image_tensor = tf.expand_dims(image_tensor, axis=0)  # Agregar una dimensión para el batch
data_tf=model.predict(image_tensor)

In [ ]:
data_tf=np.squeeze(data_tf)
data_tf.shape

## Representación de los pixeles de nuestra imágen en relación a su color 
* Usamos cada canal como punto de representación

<img src="../img/Compress Image.png" width="800">

In [ ]:
def plot_pixels(data, title, colors=None, N=10000):
    if colors is None:
        colors = data
    #print(data)        
    #Se crea una instancia de RandomState de NumPy con una semilla de valor 0. 
    #Esto se utiliza para generar números aleatorios reproducibles en la siguiente línea.
    rng = np.random.RandomState(0)
    i = rng.permutation(data.shape[0])[:N]
    #print(i)
    
    #Se genera un conjunto de índices aleatorios a partir de la forma de los datos (data) usando la función permutation del objeto RandomState. 
    #Se selecciona solo un subconjunto de N índices.
    colors = colors[i]
    R, G, B = data[i].T
    #print(colors)
    
    #print(data[i].T)
    
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    ax[0].scatter(R, G, color=colors, marker='.')
    ax[0].set(xlabel='Red', ylabel='Green', xlim=(0, 1), ylim=(0, 1))
    ax[0].set_title("Relación entre Rojo y Verde 2D")

    ax[1].scatter(R, B, color=colors, marker='.')
    ax[1].set(xlabel='Red', ylabel='Blue', xlim=(0, 1), ylim=(0, 1))
    ax[1].set_title("Relación entre Rojo y Azul 2D")

    fig.suptitle(title, size=20);
    
    # plot
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(R, G, B, color=colors, marker='.')
    ax.set(xlabel='Red', ylabel='Green', zlabel="Blue")
    ax.set_title("Relación entre RGB 3D")
    #ax.view_init(30, 85)
    plt.show()


In [ ]:
data.shape

In [ ]:
plot_pixels(data, title='Espacio latente de los colores')

## Usamos Machine Learning 

Vamos ahora a reducir de 16 millones de colores a 16.

Como se trata de un dataset grande, vamos a usar una variación del k-means llamada **mini-batch k-Means**, que funciona exactamente igual que el k-means pero con mini-batches.

Lo que hacemos es utilizar como hiperparametro de KMeans el número de cluster igual 16, porque es el número de colores al cual queremos reducir. Esto significa que todos nuestros pixeles quedrán arupados en 16 grupos.

In [ ]:
import warnings; warnings.simplefilter('ignore')  # Fix NumPy issues.
from sklearn.cluster import MiniBatchKMeans

clusters=16

#Creamos la instancia del algortimo usando los hiperparametros
kmeans = MiniBatchKMeans(n_clusters=clusters, random_state=42, batch_size=32)

#Entrenamos
kmeans.fit(data)

#Obtenemos los colores que tienen los centroides
new_colors = kmeans.cluster_centers_[kmeans.predict(data)]

#Volvemos a visualiar las gráficas pero unicamente pasamos 16 colores de clasificación
plot_pixels(data, colors=new_colors, title="Reducir espacio latente a " + str(clusters) + " colores")

Como se puede observar se han reducido los colores a 16, ahora simplemente nos queda reconstruir la imágen y colocar cada pixel en su sitio.

In [ ]:
cuadro_recolored = new_colors.reshape(cuadro.shape)

fig, ax = plt.subplots(1, 2, figsize=(16, 6), subplot_kw=dict(xticks=[], yticks=[]))
fig.subplots_adjust(wspace=0.05)
ax[0].imshow(cuadro)
ax[0].set_title('Imágen Original', size=16)
ax[1].imshow(cuadro_recolored)
ax[1].set_title('Imágen en ' + str(clusters) + ' colores', size=16);

## Comprobamos el espacio que nos ocupa en memoria la nueva imágen.

In [ ]:
import sys
print('Sin comprimir:', sys.getsizeof(cuadro))
print('Comprimida:', sys.getsizeof(cuadro_recolored))
print('Factor:', sys.getsizeof(cuadro)/sys.getsizeof(cuadro_recolored))

print(f"Es indiscutible que se pierde calidad, pero pensad que acabamos de conseguir comprimir la imagen con un factor de {sys.getsizeof(cuadro)/sys.getsizeof(cuadro_recolored)}")

**Referencias**

*   https://jakevdp.github.io/PythonDataScienceHandbook/05.11-k-means.html
*   https://jakevdp.github.io/PythonDataScienceHandbook/05.10-manifold-learning.html
*   https://www.oreilly.com/learning/an-illustrated-introduction-to-the-t-sne-algorithm
